In [35]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [36]:
news_rss_url = {
    '매일경제': 'https://www.mk.co.kr/rss/50200011/',
    '한국경제': 'https://www.hankyung.com/feed/finance'
}

In [37]:
class Article :
    def __init__(self, *, media, datetime, title, content) :
        self.media = media
        self.datetime = datetime
        self.title = title
        self.content = content

    # def __repr__(self):
    #     return f"Article(media='{self.media}', datetime='{self.datetime}', title='{self.title}', content='{self.content}'"


requests.get으로 요청했을 때 res.content가 b''로 나타나는 경우는 여러 가지 이유가 있을 수 있습니다. 특히, 해당 웹사이트가 특정 조건을 충족하지 않으면 콘텐츠를 제공하지 않도록 설정되어 있을 수 있습니다. 다음은 가능한 원인과 해결 방법입니다.

차단된 요청:
웹사이트에서 봇이나 스크래퍼의 접근을 차단할 수 있습니다. 이 경우, User-Agent 헤더를 추가하여 요청을 브라우저에서 보낸 것처럼 보이게 해야 할 수 있습니다.

In [98]:



def get_news_rss(media,url) :

    article_list = []
    title_list = []      # 기사 제목 목록
    link_list = []       # 기사 링크 목록
    news_data = []       # 기사 본문 목록
    news_datetime = []   # 기사 작성일시 목록

    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
    }
    news_rss = requests.get(url, headers=headers)
    news_rss_soup = BeautifulSoup(news_rss.content,'xml')

    # 기사 제목 수집
    title_list = news_rss_soup.select('item > title')
    title_list = [title.text for title in title_list]

    # 기사 상세링크 수집
    link_list = news_rss_soup.select('item > link')
    link_list = [link.text for link in link_list]

    # 기사 내용, 기사 작성일시 수집
    for link in link_list :
        news_res = requests.get(link, headers=headers)
        news_content_soup = BeautifulSoup(news_res.content, 'lxml')

        if media == '매일경제' :
            news_content = news_content_soup.select_one('.news_cnt_detail_wrap')
            news_data.append(news_content.text)
            news_datetime.append(news_content_soup.select_one('.time_area .registration dd').text)

        elif media == '한국경제' : 
            news_content = news_content_soup.select_one('#articletxt')
            if news_content:
                news_data.append(news_content.text)
                datetime_element = news_content_soup.select_one('.datetime > .item > .txt-date')
                news_datetime.append(datetime_element.text)
            else:
                print(link)
                continue
         
        # 기사 목록
    for item in zip(title_list, news_data, news_datetime) : 
        article = Article(media = media, title = item[0], content=item[1], datetime = item[2])
        article_list.append(article)
    return article_list


In [99]:
# 모든 언론사 기사 합치기

all_article_list=[]
for media,url in news_rss_url.items():
    print(media,url)
    list_data = get_news_rss(media, url)
    all_article_list.append(list_data)
print(len(all_article_list))

매일경제 https://www.mk.co.kr/rss/50200011/
한국경제 https://www.hankyung.com/feed/finance
2


In [100]:
result = np.array(all_article_list).flatten().tolist()
print(len(result))
print(type(result))

100
<class 'list'>


In [102]:
data = {
    '언론사': [article.media for article in result ],
    '제목': [article.title for article in result ],
    '내용': [article.content for article in result ],
    '작성일시': [article.datetime for article in result ],    
}

In [103]:
df = pd.DataFrame(data)
df.head(3)

,언론사,제목,내용,작성일시
0,매일경제,토스증권 커뮤니티서 가장 많은 관심 받은 종목은 ‘엔비디아’…2·3위는?,\n 사진 확대 토스증권 2024년 커뮤니티 결산 이미지. [사진 출...,2024-12-26 14:00:29
1,매일경제,같은 조선株여도 갈린 초고수의 선택…삼성重은 매수·현대重은 매도 [주식 초고수는 지금],\n주식 투사 수익률 상위 1% 초고수들은 26일 오전 9시 30분 기준 HD현대일...,2024-12-26 13:52:54
2,매일경제,“美 대선 등 대내외 불확실성 높았던 한 해”…코스닥 공모 금액도 3년째 줄어,\n 사진 확대 한국거래소 황소상. [사진 제공 = 한국거래소] 코...,2024-12-26 13:52:14


In [104]:
# 날짜는 파이썬 표준 라이브러리 datetime 사용
import datetime
# 오늘 날짜 가져오기
today = datetime.datetime.today()
today = today.strftime('%Y-%m-%d')
today

'2024-12-26'

In [105]:
file_name = f'news_{today}.csv'
file_name

'news_2024-12-26.csv'

In [106]:
df.to_csv(file_name,index=False)